In [0]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, roc_curve, confusion_matrix
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import label_binarize

In [0]:
!pip install qiskit

In [0]:
import qiskit
from qiskit import Aer
from qiskit import IBMQ
from qiskit import BasicAer
from qiskit.ml.datasets import *
from qiskit.circuit.library import ZZFeatureMap
from qiskit.aqua.utils import split_dataset_to_data_and_labels, map_label_to_class_name
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import QSVM

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
data = pd.read_feather('/content/drive/My Drive/Colab Notebooks/LumAB Classification/LuminalAB Data')

In [6]:
data.head()

,cancer,cnv_ACAP3,cnv_ACTRT2,cnv_AGRN,cnv_ANKRD65,cnv_ATAD3A,cnv_ATAD3B,cnv_ATAD3C,cnv_AURKAIP1,cnv_B3GALT6,cnv_C1orf159,cnv_C1orf170,cnv_C1orf222,cnv_C1orf233,cnv_C1orf86,cnv_CALML6,cnv_CCNL2,cnv_CDK11A,cnv_CDK11B,cnv_CPSF3L,cnv_DDX11L1,cnv_DVL1,cnv_FAM132A,cnv_FAM138A,cnv_FAM213B,cnv_FAM41C,cnv_FAM87B,cnv_GABRD,cnv_GLTPD1,cnv_GNB1,cnv_HES4,cnv_HES5,cnv_ISG15,cnv_KLHL17,cnv_LINC00115,cnv_LINC00982,cnv_MIB2,cnv_MIR200A,cnv_MIR200B,cnv_MIR429,...,stv_UPRT,stv_USP26,stv_USP27X,stv_USP51,stv_USP9X,stv_UTP14A,stv_UXT,stv_VCX,stv_VCX3A,stv_VSIG1,stv_VSIG4,stv_WAS,stv_WDR13,stv_WDR44,stv_WDR45,stv_WNK3,stv_XG,stv_XK,stv_XPNPEP2,stv_ZBED1,stv_ZBTB33,stv_ZC3H12B,stv_ZC4H2,stv_ZCCHC13,stv_ZCCHC16,stv_ZCCHC18,stv_ZDHHC15,stv_ZDHHC9,stv_ZFP92,stv_ZMYM3,stv_ZNF185,stv_ZNF280C,stv_ZNF41,stv_ZNF449,stv_ZNF630,stv_ZNF645,stv_ZNF674,stv_ZNF711,stv_ZXDA,stv_ZXDB
0,Luminal A,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,...,-0.063246,-0.063246,-0.063246,-0.063246,-0.169385,-0.109987,-0.063246,-0.120145,-0.089623,-0.063246,-0.12726,-0.063246,-0.063246,-0.135117,-0.063246,-0.123477,-0.063246,-0.063246,-0.063246,-0.063246,-0.063246,-0.156499,-0.063246,-0.063246,-0.089623,-0.063246,-0.109987,-0.063246,-0.063246,-0.12726,-0.063246,-0.063246,-0.089623,-0.063246,-0.063246,-0.063246,-0.063246,-0.089623,-0.089623,-0.063246
1,Luminal A,0.671560,0.671560,0.671560,0.671560,0.671560,0.671560,0.671560,0.671560,0.671560,0.671560,0.671560,0.671560,0.671560,0.671560,0.671560,0.671560,0.671560,0.671560,0.671560,0.671560,0.671560,0.671560,0.671560,0.671560,0.671560,0.671560,0.671560,0.671560,0.671560,0.671560,0.671560,0.671560,0.671560,0.671560,0.671560,0.671560,0.671560,0.671560,0.671560,...,-0.063246,-0.063246,-0.063246,-0.063246,-0.169385,-0.109987,-0.063246,-0.120145,-0.089623,-0.063246,-0.12726,-0.063246,-0.063246,-0.135117,-0.063246,-0.123477,-0.063246,-0.063246,-0.063246,-0.063246,-0.063246,6.364275,-0.063246,-0.063246,-0.089623,-0.063246,-0.109987,-0.063246,-0.063246,-0.12726,-0.063246,-0.063246,-0.089623,-0.063246,-0.063246,-0.063246,-0.063246,-0.089623,-0.089623,-0.063246
2,Luminal B,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,...,-0.063246,-0.063246,-0.063246,-0.063246,-0.169385,-0.109987,-0.063246,-0.120145,-0.089623,-0.063246,-0.12726,-0.063246,-0.063246,-0.135117,-0.063246,-0.123477,-0.063246,-0.063246,-0.063246,-0.063246,-0.063246,-0.156499,-0.063246,-0.063246,-0.089623,-0.063246,-0.109987,-0.063246,-0.063246,-0.12726,-0.063246,-0.063246,-0.089623,-0.063246,-0.063246,-0.063246,-0.063246,-0.089623,-0.089623,-0.063246
3,Luminal A,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,-1.133708,...,-0.063246,-0.063246,-0.063246,-0.063246,-0.169385,-0.109987,-0.063246,-0.120145,-0.089623,-0.063246,-0.12726,-0.063246,-0.063246,-0.135117,-0.063246,-0.123477,-0.063246,-0.063246,-0.063246,-0.063246,-0.063246,-0.156499,-0.063246,-0.063246,-0.089623,-0.063246,-0.109987,-0.063246,-0.063246,-0.12726,-0.063246,-0.063246,-0.089623,-0.063246,-0.063246,-0.063246,-0.063

In [0]:
data['cancer'] = label_binarize(data['cancer'], classes=list(set(data['cancer'])))

X = data.drop('cancer', axis=1)
y = data['cancer']

In [0]:
pca = PCA(n_components=10)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
X_train_reduced = pca.fit_transform(X_train)

In [0]:
reduced_train_data = pd.DataFrame(data=X_train_reduced, columns=['PC {}'.format(i) for i in range(1, 11)])
reduced_train_data = reduced_train_data.join(pd.DataFrame.reset_index(pd.DataFrame(y_train), drop=True))
training_data = {k: np.asarray(g.iloc[:, :-1]) for k, g in reduced_train_data.groupby('cancer')}

In [0]:
training_data

In [0]:
X_test_reduced = pca.transform(X_test)
reduced_test_data = pd.DataFrame(data=X_test_reduced, columns=['PC {}'.format(i) for i in range(1, 11)])
reduced_test_data = reduced_test_data.join(pd.DataFrame.reset_index(pd.DataFrame(y_test), drop=True))
test_data = {k: np.asarray(g.iloc[:, :-1]) for k, g in reduced_test_data.groupby('cancer')}

In [0]:
seed = 10598
feature_dim = 10

feature_map = ZZFeatureMap(feature_dimension=feature_dim, reps=2, entanglement='linear')
qsvm = QSVM(feature_map, training_data, test_data)

backend = BasicAer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=seed, seed_transpiler=seed)

In [0]:
result = qsvm.run(quantum_instance)

In [33]:
print("Quantum accuracy on test set: {0}%".format(round(result['testing_accuracy']*100, 2)))

Quantum accuracy on test set: 64.1%
